In [1]:
from vsrc.constants import DATA
from ultralytics import YOLO
from sparseml.pytorch.optim import ScheduledModifierManager
from sparseml.pytorch.utils import get_prunable_layers
from torch.optim import AdamW
from os import path, listdir, makedirs
from shutil import rmtree, copy
import random
import time
import cv2

WARNING  Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View settings with 'yolo settings' or at 'C:\Users\Shaheen\AppData\Roaming\Ultralytics\settings.yaml'
Update settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
BASE_MODEL = "./models/final/classifier.pt"
RECIPE_PATH = "./models/recipes/recipe.yaml"

model = YOLO(BASE_MODEL)
# model.model.to("cuda")

In [31]:
# Constants
IMAGE_SIZE = 640 # x 480
PROJECT_NAME = "logs"
RECIPE_NAME = "sparse_obb"
TRAIN_RUN = f"{RECIPE_NAME}_run-{time.strftime('%Y%m%d-%H%M%S')}"
DATA_PATH = "./datasets/full"

# Model
MODEL_PATH = "./models/pretrained/yolov8n-obb-dotav1.pt"
TEST_PATH = "./models/recipes/dataset_desc_test.yaml"
TRAIN_PARAM = {
    # Model definition
    'data': "./models/recipes/dataset_desc.yaml",
    'resume': False,
    'device': '0',
    'pretrained': True,
    # Names
    'project' : PROJECT_NAME,
    'name': TRAIN_RUN,
    # Training Parameters
    'batch': -1,
    'imgsz': IMAGE_SIZE,
    'epochs': 1,
    'cos_lr': False,
    # "lr0": 0.05,
    # Augmentation
    'hsv_h': 0.05, # Higher than default for resistor
    'hsv_s': 0.3, # Colours should not change too much
    'hsv_v': 0.2, # Colours should not change too much
    'degrees': 180, # Rotation
    'translate': 0.1, # Translation
    'scale': 0.8, # Scaling - camera is always at the same distance
    'shear': 10.0, # Shearing
    'perspective': 0.0, # Perspective
    'flipud': 0.5, # Flip up-down
    'fliplr': 0.5, # Flip left-right
    'mosaic': 0.5, # Mosaic
    'mixup': 0.0, # Mixup
    'copy_paste': 0.0, # Copy-paste
    'crop_fraction': 1.0, # Crop fraction
    # Loss weights
    # 'cls' : 1.0, # Class
    # 'box' : 4.0, # Box accuracy
    # 'dfl' : 1.5, # Help manage unbalanced classes
    # Post parameters
    'save': False,
    'save_period': 5,
    'plots': False,
    # Misc
    'verbose': False,
}

In [32]:
metrics = model.train(**TRAIN_PARAM)

New https://pypi.org/project/ultralytics/8.2.31 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.20  Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16384MiB)
engine\trainer: task=obb, mode=train, model=./models/final/classifier.pt, data=./models/recipes/dataset_desc.yaml, epochs=1, time=None, patience=100, batch=-1, imgsz=640, save=False, save_period=5, cache=False, device=0, workers=8, project=logs, name=sparse_obb_run-20240611-030544, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=Fa

train: Scanning C:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets\full\current\labels\train.cache... 682 images, 0 backgrounds, 0 corrupt: 100%|██████████| 682/682 [00:00<?, ?it/s]
val: Scanning C:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets\full\current\labels\val.cache... 195 images, 0 backgrounds, 0 corrupt: 100%|██████████| 195/195 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000667, momentum=0.9) with parameter groups 63 weight(decay=0.0), 73 weight(decay=0.00034375), 72 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to logs\sparse_obb_run-20240611-030544
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1      6.52G      1.014     0.5988      1.539         37        640: 100%|██████████| 16/16 [00:08<00:00,  1.88it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:02<00:00,  2.29it/s]

                   all        195        195       0.99          1      0.995      0.798



1 epochs completed in 0.005 hours.
Optimizer stripped from logs\sparse_obb_run-20240611-030544\weights\last.pt, 6.7MB
Optimizer stripped from logs\sparse_obb_run-20240611-030544\weights\best.pt, 6.7MB

Validating logs\sparse_obb_run-20240611-030544\weights\best.pt...
Ultralytics YOLOv8.2.20  Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16384MiB)
YOLOv8n-obb summary (fused): 187 layers, 3079364 parameters, 0 gradients, 8.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  3.29it/s]

                   all        195        195       0.99          1      0.995      0.799
Speed: 0.3ms preprocess, 2.2ms inference, 0.0ms loss, 2.4ms postprocess per image


In [40]:
metrics.results_dict

{'metrics/precision(B)': 0.9899909996370566,
 'metrics/recall(B)': 1.0,
 'metrics/mAP50(B)': 0.995,
 'metrics/mAP50-95(B)': 0.7986636128091695,
 'fitness': 0.8182972515282526}

In [27]:
optimizer = AdamW(lr=0.000588, params=model.model.parameters())
manager = ScheduledModifierManager.from_yaml(RECIPE_PATH)
sparseModel = manager.modify(model.model, steps_per_epoch=1, optimizer=optimizer)

In [25]:
for (name, layer) in get_prunable_layers(model):
    print(f"{name}")

model.model.0.conv
model.model.1.conv
model.model.2.cv1.conv
model.model.2.cv2.conv
model.model.2.m.0.cv1.conv
model.model.2.m.0.cv2.conv
model.model.3.conv
model.model.4.cv1.conv
model.model.4.cv2.conv
model.model.4.m.0.cv1.conv
model.model.4.m.0.cv2.conv
model.model.4.m.1.cv1.conv
model.model.4.m.1.cv2.conv
model.model.5.conv
model.model.6.cv1.conv
model.model.6.cv2.conv
model.model.6.m.0.cv1.conv
model.model.6.m.0.cv2.conv
model.model.6.m.1.cv1.conv
model.model.6.m.1.cv2.conv
model.model.7.conv
model.model.8.cv1.conv
model.model.8.cv2.conv
model.model.8.m.0.cv1.conv
model.model.8.m.0.cv2.conv
model.model.9.cv1.conv
model.model.9.cv2.conv
model.model.12.cv1.conv
model.model.12.cv2.conv
model.model.12.m.0.cv1.conv
model.model.12.m.0.cv2.conv
model.model.15.cv1.conv
model.model.15.cv2.conv
model.model.15.m.0.cv1.conv
model.model.15.m.0.cv2.conv
model.model.16.conv
model.model.18.cv1.conv
model.model.18.cv2.conv
model.model.18.m.0.cv1.conv
model.model.18.m.0.cv2.conv
model.model.19.conv

In [ ]:
def train_one_epoch(model, sparseModel, optimizer, data):
    model.train()
    sparseModel.train()
    for i, (img, target) in enumerate(data):
        img = img.to("cuda")
        target = target.to("cuda")
        optimizer.zero_grad()
        loss = sparseModel(img, target)
        loss.backward()
        optimizer.step()
        print(f"Batch {i+1}/{len(data)} Loss: {loss.item()}")
    return loss

In [3]:
! sparseml.ultralytics.train --recipe "./models/recipes/sparsify_ex.yaml" --model "./models/pretrained/yolov8n-obb-dotav1.pt" --data "./models/recipes/dataset_desc.yaml"

Traceback (most recent call last):
  File "C:\Users\Shaheen\anaconda3\envs\fyp\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Shaheen\anaconda3\envs\fyp\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\Shaheen\Anaconda3\envs\fyp\Scripts\sparseml.ultralytics.train.exe\__main__.py", line 7, in <module>
  File "C:\Users\Shaheen\anaconda3\envs\fyp\lib\site-packages\click\core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "C:\Users\Shaheen\anaconda3\envs\fyp\lib\site-packages\click\core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "C:\Users\Shaheen\anaconda3\envs\fyp\lib\site-packages\click\core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "C:\Users\Shaheen\anaconda3\envs\fyp\lib\site-packages\click\core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "C:\Users\Shaheen\anaconda3\envs\fyp\lib\sit